In [21]:
import os
import json
import random
from datetime import datetime, timedelta

import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import models
import torchvision.transforms as T

In [4]:
with open('train_labels.json', 'r') as file:
    train_data = json.load(file)
with open('test_labels.json', 'r') as file:
    test_data = json.load(file)

In [5]:
for k, v in train_data.items():
    assert v['start'] < v['end'], v

In [6]:
for k, v in test_data.items():
    assert v['start'] < v['end'], v

In [7]:
len(train_data), len(test_data)

(80, 45)

In [13]:
to_time = lambda s: datetime.strptime(s, "%H:%M:%S")

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [22]:
class ClsDataset(torch.utils.data.Dataset):
    def __init__(self, folder, data, mode='train'):
        assert mode in ['train', 'test', 'val'], "Check your Dataset mode!"
        
        interval_sec = 1
        total_minutes = 13
        
        if mode == 'train':
            # self.trans = T.Compose([
            #     # Some augmentations
            #     T.ToTensor(),
            #     T.Resize((256, 256))
            # ])
            self.trans = models.ResNet34_Weights.IMAGENET1K_V1.transforms()
        else:
            # self.trans = T.Compose([
            #     T.ToTensor(),
            #     T.Resize((256, 256))
            # ])
            self.trans = models.ResNet34_Weights.IMAGENET1K_V1.transforms()

        self.images = []
        self.labels = []
        fs = [ f for f in os.listdir(folder) if not f.startswith('.') ]
        for f in tqdm(fs):
            video = os.listdir(os.path.join(folder, f))[0]
            cap = cv2.VideoCapture(os.path.join(folder, f, video))
            if not cap.isOpened():
                cap.release()
                print(f"Ошибка при загрузке видео '{video_path}'!")
                continue
            
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_interval = int(fps * interval_sec)
            
            label = data[f]
            # print(label)
            start_frame = int((time_to_seconds(label['start']) + 1) * fps)
            end_frame = int(time_to_seconds(label['end']) * fps)
            duration_frames = end_frame - start_frame
            
            extraction_points = [
                (start_frame, end_frame),
                (end_frame + duration_frames, end_frame + 2 * duration_frames),
                (end_frame + 2 * duration_frames, end_frame + 3 * duration_frames)
            ]
            
            
            for ind, (start, end) in enumerate(extraction_points):
                saved_frames = 0
                cap.set(cv2.CAP_PROP_POS_FRAMES, start)
                current_frame = start
                
                while current_frame < end:
                    ret, frame = cap.read()
                    if not ret:
                        break
                    
                    if current_frame % frame_interval == 0:
                        self.images.append(
                            self.trans(
                                torch.tensor(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).permute(2, 0, 1)
                            )
                        )
                        self.labels.append(int(ind==0))
                        saved_frames += 1
                    
                    current_frame += 1
                # print(f"{f}; saved frames: {saved_frames}")
            
            cap.release()

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [23]:
test_dataset = ClsDataset('test', test_data, mode='test')

  0%|          | 0/45 [00:00<?, ?it/s]

In [24]:
train_dataset = ClsDataset('train', train_data, mode='train')

  0%|          | 0/80 [00:00<?, ?it/s]

In [25]:
batch_size=64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                              pin_memory=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True,
                                              pin_memory=True, num_workers=4)

In [26]:
for x_batch, y_batch in train_dataloader:
    print(x_batch.shape)
    print(y_batch.shape)
    break

torch.Size([64, 3, 224, 224])
torch.Size([64])


In [27]:
resnet34 = models.resnet34(weights=models.ResNet34_Weights.DEFAULT).to(device)
in_ftr = resnet34.fc.in_features
resnet34.fc = nn.Linear(in_ftr, 2).to(device)

In [28]:
lr = 1e-5
epochs = 7

model = resnet34
opt = torch.optim.AdamW(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss(reduction='sum')
scheduler = torch.optim.lr_scheduler.LinearLR(opt, 0.1, 1, 7)
# scheduler = None

In [29]:
from math import ceil
history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': [], 'lr': []}

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")

    history['lr'].append(opt.param_groups[0]['lr'])
    model.train()
    train_loss = 0
    train_acc = 0
    num_samples = 0
    for idx, (X_batch, y_batch) in enumerate(tqdm(train_dataloader, desc="Train")):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        pred = model(X_batch)
        loss = criterion(pred, y_batch)

        opt.zero_grad()
        loss.backward()
        opt.step()

        train_loss += loss.item()
        train_acc += (torch.argmax(pred, dim=-1) == y_batch).sum().item()
        num_samples += len(X_batch)
            
    train_loss /= num_samples
    train_acc /= num_samples
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)


    model.eval()
    val_loss = 0
    val_acc = 0
    num_samples = 0
    with torch.no_grad():
        for idx, (X_batch, y_batch) in enumerate(tqdm(test_dataloader, desc="Validation")):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            pred = model(X_batch)
            loss = criterion(pred, y_batch)

        
            val_loss += loss.item()
            val_acc += (torch.argmax(pred, dim=-1) == y_batch).sum().item()
            num_samples += len(X_batch)

    val_loss /= num_samples
    val_acc /= num_samples
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)

    if scheduler is not None:
        try:
            scheduler.step()
        except:
            scheduler.step(history[''])
    
    print(f"Training Loss: {history['train_loss'][-1]:.4f}, Training Accuracy: {history['train_acc'][-1]:.4f}")
    print(f"Testing Loss: {history['val_loss'][-1]:.4f}, Testing Accuracy: {history['val_acc'][-1]:.4f}")
    print(f"Learning Rate: {history['lr'][-1]:.3e}")
    
    print()

Epoch 1/7


Train:   0%|          | 0/51 [00:00<?, ?it/s]

Validation:   0%|          | 0/21 [00:00<?, ?it/s]

Training Loss: 0.7427, Training Accuracy: 0.4913
Testing Loss: 0.6495, Testing Accuracy: 0.6328
Learning Rate: 1.000e-06

Epoch 2/7


Train:   0%|          | 0/51 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/21 [00:00<?, ?it/s]

Training Loss: 0.1233, Training Accuracy: 0.9676
Testing Loss: 0.1630, Testing Accuracy: 0.9616
Learning Rate: 7.429e-06

Epoch 7/7


Train:   0%|          | 0/51 [00:00<?, ?it/s]

Validation:   0%|          | 0/21 [00:00<?, ?it/s]

Training Loss: 0.0796, Training Accuracy: 0.9832
Testing Loss: 0.1528, Testing Accuracy: 0.9571
Learning Rate: 8.714e-06



In [32]:
from copy import deepcopy
cls_model = deepcopy(model)

In [238]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [201]:
class RegDataset(torch.utils.data.Dataset):
    def __init__(self, folder, data, mode='train'):
        assert mode in ['train', 'test', 'val'], "Check your Dataset mode!"
        self.mode = mode
        cls_model.avgpool = nn.Identity().to(device)
        cls_model.fc = nn.Identity().to(device)
        
        interval_sec = 1
        total_minutes = 13

        self.backbone_trans = models.ResNet34_Weights.IMAGENET1K_V1.transforms()
        self.trans = T.Compose([
            T.RandomHorizontalFlip(p=0.5),
            T.RandomRotation(15),
        ])

        self.videos = []
        self.labels = []
        fs = [ f for f in os.listdir(folder) if not f.startswith('.') ]
        with torch.no_grad():
            for f in tqdm(fs):
                video_name = os.listdir(os.path.join(folder, f))[0]
                cap = cv2.VideoCapture(os.path.join(folder, f, video_name))
                video = []
                if not cap.isOpened():
                    cap.release()
                    print(f"Ошибка при загрузке видео '{video_path}'!")
                    continue
                
                fps = cap.get(cv2.CAP_PROP_FPS)
                frame_interval = int(fps * interval_sec)
                end = 512
                
                label = data[f]
                start = (datetime.strptime(label['start'], '%H:%M:%S') -
                            datetime.strptime('0:00:00', '%H:%M:%S')).seconds / end
                duration = (datetime.strptime(label['end'], '%H:%M:%S') -
                            datetime.strptime(label['start'], '%H:%M:%S')).seconds / end
                self.labels.append((start, duration))
                if start + duration > end:
                    continue
                saved_frames = 0
                current_frame = 0
                
                while saved_frames < end:
                    ret, frame = cap.read()
                    if not ret:
                        break
                    
                    if current_frame % frame_interval == 0:
                        video.append(
                            cls_model(
                                self.backbone_trans(
                                    torch.tensor(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).permute(2, 0, 1)
                                ).unsqueeze(0).to(device)
                            ).to('cpu').squeeze().view(512, 7, 7)
                        )
                        saved_frames += 1
                    current_frame += 1
                video = torch.stack(video)
                if video.shape[0] == end:
                    self.videos.append(video)
                # print(f"{f}; saved frames: {saved_frames}")
                
                cap.release()

    def __len__(self):
        return len(self.videos)
    
    def __getitem__(self, idx):
        video = self.videos[idx]
        if self.mode == 'train':
            video = self.trans(video)
        return video, torch.tensor(self.labels[idx], dtype=torch.float32)

In [149]:
test_reg_dataset = RegDataset('test', test_data, mode='test')

  0%|          | 0/45 [00:00<?, ?it/s]

In [202]:
train_reg_dataset = RegDataset('train', train_data)

  0%|          | 0/80 [00:00<?, ?it/s]

[NULL @ 0x963bc800] Invalid NAL unit size (12559 > 4470).
[NULL @ 0x963bc800] missing picture in access unit with size 4486
[h264 @ 0xd2ebdcc0] Invalid NAL unit size (12559 > 4470).
[h264 @ 0xd2ebdcc0] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x8c331e80] stream 0, offset 0x12c01f99: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x8c331e80] stream 1, offset 0x12c06937: partial file


In [239]:
batch_size=16
train_reg_dataloader = DataLoader(train_reg_dataset, batch_size=batch_size, shuffle=True,
                                              pin_memory=True, num_workers=4)
test_reg_dataloader = DataLoader(test_reg_dataset, batch_size=batch_size, shuffle=False,
                                              pin_memory=True, num_workers=4)

In [343]:
class TimeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv3d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm3d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            
            nn.Conv3d(512, 256, kernel_size=3, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(),
            nn.Dropout(0.4),
            
            nn.Conv3d(256, 256, kernel_size=3, padding=1),
            nn.AdaptiveAvgPool3d((1, 1, 1)),
            nn.Dropout(0.4)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        x = self.features(x.permute(0, 2, 1, 3, 4))
        x = x.view(x.size(0), -1)
        return self.classifier(x)

In [344]:
def weighted_mse_loss(y_pred, y_true, weights=torch.tensor([1, 2]).to(device)):
    return ((weights * (y_pred - y_true) ** 2).sum())

In [345]:
lr = 1e-4
epochs = 18

model = TimeNet().to(device)
opt = torch.optim.AdamW(model.parameters(), lr=lr)
criterion = weighted_mse_loss
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'min', factor=0.5, patience=5, min_lr=1e-6)

In [346]:
torch.set_printoptions(sci_mode=False)

In [347]:
from math import ceil
history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': [], 'lr': []}

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")

    history['lr'].append(opt.param_groups[0]['lr'])
    model.train()
    train_loss = 0
    num_samples = 0
    for idx, (X_batch, y_batch) in enumerate(tqdm(train_reg_dataloader, desc="Train")):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device) * 512
        pred = model(X_batch)
        if idx == 0:
            print(pred[:5])
            print(y_batch[:5])
        loss = criterion(pred, y_batch)

        opt.zero_grad()
        loss.backward()
        opt.step()

        train_loss += loss.item()
        num_samples += len(X_batch)
            
    train_loss /= num_samples
    history['train_loss'].append(train_loss)


    model.eval()
    val_loss = 0
    num_samples = 0
    with torch.no_grad():
        for idx, (X_batch, y_batch) in enumerate(tqdm(test_reg_dataloader, desc="Validation")):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device) * 512
            pred = model(X_batch)
            loss = criterion(pred, y_batch)

        
            val_loss += loss.item()
            num_samples += len(X_batch)

    val_loss /= num_samples
    history['val_loss'].append(val_loss)

    if scheduler is not None:
        try:
            scheduler.step()
        except:
            scheduler.step(history['train_loss'][-1])
    
    print(f"Training Loss: {history['train_loss'][-1]:.4f}")
    print(f"Testing Loss: {history['val_loss'][-1]:.4f}")
    print(f"Learning Rate: {history['lr'][-1]:.3e}")
    
    print()

Epoch 1/18


Train:   0%|          | 0/5 [00:00<?, ?it/s]

tensor([[-0.2114, -0.0125],
        [-0.2257, -0.0717],
        [-0.0889, -0.0242],
        [-0.1251, -0.0009],
        [-0.2088, -0.0605]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[260.,   7.],
        [ 17.,   5.],
        [ 67.,   3.],
        [ 66.,   4.],
        [104.,   4.]], device='cuda:0')


Validation:   0%|          | 0/3 [00:00<?, ?it/s]

Training Loss: 36966.9150
Testing Loss: 8936.3763
Learning Rate: 1.000e-04

Epoch 2/18


...

Training Loss: 7678.1516
Testing Loss: 33781.5049
Learning Rate: 1.000e-04

Epoch 18/18


Train:   0%|          | 0/5 [00:00<?, ?it/s]

tensor([[156.1927,  36.0975],
        [108.1979,  17.3564],
        [102.0363,   4.3859],
        [ 63.5703,   7.2555],
        [ 43.9919,   3.8690]], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([[ 16.,   6.],
        [ 50.,   5.],
        [ 36.,   6.],
        [  5.,  10.],
        [104.,   4.]], device='cuda:0')


Validation:   0%|          | 0/3 [00:00<?, ?it/s]

Training Loss: 6655.9268
Testing Loss: 12836.0736
Learning Rate: 1.000e-04



In [348]:
for idx, (X_batch, y_batch) in enumerate(tqdm(test_reg_dataloader, desc="Validation")):
    X_batch, y_batch = X_batch.to(device), y_batch.to(device) * 512
    pred = model(X_batch)
    for r1, r2 in zip(pred, y_batch):
        print(r1.cpu().detach().numpy(), r2.cpu().detach().numpy())
    print()

Validation:   0%|          | 0/3 [00:00<?, ?it/s]

[32.076027  5.056288] [113.   7.]
[103.29551  16.20145] [ 5. 10.]
[55.501007  8.720691] [167.  23.]
[110.36651   17.309397] [ 5. 10.]
[19.65943    3.1132293] [36.  6.]
[166.79456   26.138346] [ 5. 10.]
[35.41662    5.5795965] [43.  4.]
[129.88368  20.36243] [ 5. 10.]
[80.15578  12.580447] [ 5. 11.]
[38.372475  6.04299 ] [51. 35.]
[21.227554   3.3582954] [50.  5.]
[120.67631   18.921507] [ 5. 10.]
[18.370466   2.9117932] [6. 2.]
[91.85646  14.412247] [ 5. 10.]
[164.30463   25.749956] [ 5. 10.]
[143.84273   22.547192] [ 5. 11.]

[117.225815  18.381618] [ 5. 11.]
[127.62823   20.007856] [ 5. 11.]
[103.49941  16.23243] [ 5. 11.]
[97.640686 15.317229] [ 5. 11.]
[16.44832    2.6104724] [6. 3.]
[117.59487  18.43973] [ 5. 10.]
[132.2519    20.733004] [10. 10.]
[7.5699472 1.2205622] [6. 3.]
[194.02412   30.399662] [10. 10.]
[329.79236  51.64201] [582.  43.]
[119.74803   18.777023] [ 5. 11.]
[132.15833   20.718271] [ 5. 11.]
[112.494705  17.63969 ] [ 5. 10.]
[155.08266  24.30567] [ 5. 15.]
[23.4

In [349]:
for idx, (X_batch, y_batch) in enumerate(tqdm(train_reg_dataloader, desc="Train")):
    X_batch, y_batch = X_batch.to(device), y_batch.to(device) * 512
    pred = model(X_batch)
    for r1, r2 in zip(pred, y_batch):
        print(r1.cpu().detach().numpy(), r2.cpu().detach().numpy())
    print()

Validation:   0%|          | 0/5 [00:00<?, ?it/s]

[17.162     2.722292] [17.  5.]
[497.2698   77.85048] [708.  41.]
[31.346575   4.9408293] [129.  11.]
[23.72272   3.749144] [17.  5.]
[58.551647  9.200684] [95. 36.]
[30.508663   4.8109174] [50.  5.]
[51.131767   8.0378895] [188.   4.]
[86.76142  13.613669] [171.  24.]
[20.831577   3.2956407] [106.  10.]
[16.288006  2.585763] [4. 4.]
[111.69178   17.514553] [363.   8.]
[19.409601   3.0732455] [58.  4.]
[13.735755  2.185643] [17.  5.]
[27.842634  4.392887] [42.  5.]
[35.542347  5.599921] [51. 35.]
[47.79317    7.5148187] [189.  10.]

[12.519635   1.9953722] [10. 11.]
[21.62419    3.4209404] [16.  6.]
[206.45073  32.34634] [399.  43.]
[257.80804   40.381554] [459.  41.]
[7.2451243 1.169662 ] [6. 3.]
[299.6396    46.924126] [582.  43.]
[169.7551   26.60132] [393.  43.]
[84.17347  13.208206] [245.   8.]
[6.9176993 1.1187226] [6. 4.]
[8.098572  1.3031313] [6. 4.]
[31.189444  4.918494] [90. 59.]
[179.19188   28.072344] [256.   6.]
[20.260036   3.2067409] [48.  2.]
[142.93858   22.409348] [28

In [350]:
torch.save(resnet34.state_dict(), "resnet34_weights.pth")

In [351]:
torch.save(model.state_dict(), "timenet_weights.pth")